In [1]:
state = 'NE' 
year = 2020
obj_type = 'perimeter'
starting_deviation = 0.01

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of NE has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from NE_plans_2020 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [6]:
from pareto import filter_and_sort_pareto
from metrics import scores

plans_scores = [scores(G, plan, G._ideal_population, obj_type) for plan in warm_starts]
_,_,nondominated_warm_starts_plans = filter_and_sort_pareto(plans=warm_starts, upper_bounds=plans_scores, obj_type=obj_type)
print(f"Selected {len(nondominated_warm_starts_plans)} nondominated warm start plans")

Selected 0 nondominated warm start plans


In [ ]:
from epsilon_constraint import epsilon_constraint_method

(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity = 'lcut',                                             # {'lcut', 'scf', 'shir'} 
            cutoff=None,       
            verbose= True,
            warm_start_mode = 'user',                                        # {'None', 'user', 'refinement'}
            warm_starts=warm_starts,                                         # if you have user define warm starts else it is None
            starting_deviation=starting_deviation, 
            time_limit=7200, 
            sizes=None,      
            max_B=True,                                                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True   
            symmetry_breaking='orbitope',                                    # {None, 'orbitope', 'rsum'} 
            state=state,
            year=year
        )

Initially, L = 647297 and U = 660373 and k = 3.

****************************************
Trying deviation = 6538.346666666666
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 647297 and U = 660373
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical process

Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 3804 rows, 3243 columns and 12403 nonzeros
Model fingerprint: 0x7d0ecaf8
Variable types: 837 continuous, 2406 integer (2406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [8e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 1986 row

Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Xeon(R) CPU E5-2630 v4 @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  7200
FeasibilityTol  1e-07
IntFeasTol  1e-07
MIPGap  0
LazyConstraints  1

Optimize a model with 3804 rows, 3243 columns and 12403 nonzeros
Model fingerprint: 0xa4c67498
Variable types: 837 continuous, 2406 integer (2406 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+05]
  Objective range  [8e-04, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+05]
Presolve removed 1986 row

     0     0   28.05811    0  403          -   28.05811      -     -    3s
     0     0   28.13643    0  350          -   28.13643      -     -    3s
     0     0   28.15778    0  365          -   28.15778      -     -    3s
     0     0   28.15778    0  366          -   28.15778      -     -    3s
     0     0   28.15778    0  367          -   28.15778      -     -    3s
     0     0   28.15778    0  367          -   28.15778      -     -    3s
     0     2   28.16704    0  367          -   28.16704      -     -    4s
   170   194   35.29566   12  174          -   30.31022      -   117    5s
  1943  1575   42.56113   28  467          -   30.85926      -  88.1   10s
  2350  1886   41.92739   24  299          -   30.85926      -   103   15s
  4240  2782   41.51213   31  400          -   31.35844      -  98.3   20s
  9068  5232   37.34855   28   83          -   32.08001      -  82.0   25s
 13999  8599 infeasible   21               -   33.03639      -  78.9   30s
 18312 11212   36.77509  

Best objective 4.039938753888e+01, best bound 4.039938753888e+01, gap 0.0000%

User-callback calls 92167, time in user-callback 10.03 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[9, 10, 26, 43, 46, 76], [7, 16, 19, 21, 28, 44, 47, 61, 65, 69, 71], [0, 1, 2, 3, 4, 5, 6, 8, 11, 12, 13, 14, 15, 17, 18, 20, 22, 23, 24, 25, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 45, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 62, 63, 64, 66, 67, 68, 70, 72, 73, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]]

****************************************
Trying deviation = 179.4999999999612
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 653656 and U = 654014
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex orderin

Variable types: 0 continuous, 1832 integer (1832 binary)

Root relaxation: objective 2.838515e+01, 976 iterations, 0.11 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.38515    0  281          -   28.38515      -     -    0s
     0     0   28.76517    0  252          -   28.76517      -     -    0s
     0     0   28.85527    0  295          -   28.85527      -     -    0s
     0     0   28.85674    0  251          -   28.85674      -     -    0s
     0     0   28.90949    0  313          -   28.90949      -     -    1s
     0     0   28.91249    0  317          -   28.91249      -     -    1s
     0     0   28.91266    0  317          -   28.91266      -     -    1s
     0     0   28.91890    0  334          -   28.91890      -     -    1s
     0     0   28.92537    0  337          -   28.92537      -     -    1s
     0     0   28.92549    

Variable types: 0 continuous, 1832 integer (1832 binary)

Root relaxation: objective 2.838807e+01, 1141 iterations, 0.20 seconds (0.04 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   28.38807    0  281          -   28.38807      -     -    0s
     0     0   28.66852    0  335          -   28.66852      -     -    0s
     0     0   28.84895    0  339          -   28.84895      -     -    1s
     0     0   28.86385    0  362          -   28.86385      -     -    1s
     0     0   28.89765    0  361          -   28.89765      -     -    1s
     0     0   28.89800    0  371          -   28.89800      -     -    1s
     0     0   28.90198    0  366          -   28.90198      -     -    1s
     0     0   28.96368    0  345          -   28.96368      -     -    2s
     0     0   28.96432    0  355          -   28.96432      -     -    2s
     0     0   28.96432   

  StrongCG: 1
  Flow cover: 2
  GUB cover: 1
  Inf proof: 6
  Zero half: 4
  RLT: 10
  Lazy constraints: 670

Explored 218861 nodes (16414384 simplex iterations) in 460.17 seconds (90.96 work units)
Thread count was 20 (of 20 available processors)

Solution count 1: 45.8964 

Optimal solution found (tolerance 0.00e+00)
Best objective 4.589637576140e+01, best bound 4.589637576140e+01, gap 0.0000%

User-callback calls 445218, time in user-callback 19.37 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
plan = [[10, 13, 43, 44, 72, 74, 76, 77], [6, 7, 15, 25, 28, 29, 33, 47, 48, 50, 51, 60, 61, 65, 69, 71, 87, 91], [0, 1, 2, 3, 4, 5, 8, 9, 11, 12, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 27, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 45, 46, 49, 52, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 66, 67, 68, 70, 73, 75, 78, 79, 80, 81, 82, 83, 84, 85, 86, 88, 89, 90, 92]]

***************************************

 140720 87339   51.75899   55   60          -   37.24893      -  56.4  277s
 142577 88468   47.20469   45  203          -   37.29479      -  56.4  280s
 144784 89521   40.83975   45  260          -   37.32559      -  56.4  285s
 148852 90972 infeasible   47               -   37.39712      -  56.4  293s
 149948 92298   50.62704   29  304          -   37.45461      -  56.4  297s
 152307 93227   47.43868   42  192          -   37.50889      -  56.4  301s
 156414 95340   56.24022   37  348          -   37.58015      -  56.4  308s
 158554 96465   39.75553   24  239          -   37.63525      -  56.4  312s
 160771 97493   44.26762   57  102          -   37.68412      -  56.4  316s
 164883 99160   39.74297   45  367          -   37.74801      -  56.4  325s
 168457 101172   49.38234   38   57          -   37.83488      -  56.3  331s
 170236 102209   47.29479   43  218          -   37.86743      -  56.4  335s
 174436 104126   41.43981   31  433          -   37.91396      -  56.4  341s
 176156 1

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
if any(round(x[1][0], 2) != round(x[1][1], 2) for x in result):
    min_deviation = max(x[2] for x in result if round(x[1][0], 2) != round(x[1][1], 2))
else:
    min_deviation = min(x[2] for x in result)
    
print('min_deviation:' , min_deviation)

In [ ]:
from enumeration import enumerate_and_solve_k2_subproblems
from metrics import observed_deviation_persons, compute_obj
import math

epsilon = 1 / (2 * G._k)

while True:
    print(f"\n{'*' * 40}\nTrying deviation = {min_deviation}\n{'*' * 40}")
    L = math.ceil(G._ideal_population - min_deviation)
    U = math.floor(G._ideal_population + min_deviation)
    G._L = L
    G._U = U
    print(f'L = {L} and U = {U}')

    new_plans = enumerate_and_solve_k2_subproblems(G, min_deviation, L, U, G._k, obj_type = obj_type, root='Douglas')
    
    if not new_plans:
        print("No plans found. Reducing deviation or stopping.")
        break  

    enumeration_plans = {}
    for i, plan in enumerate(new_plans):
        obj_value = compute_obj(G, plan, obj_type)
        dev = observed_deviation_persons(G, plan, G._ideal_population, year=year)
        enumeration_plans[i] = [obj_value, dev]

    if obj_type in {"inverse_Polsby_Popper", "cut_edges", "perimeter"}:
        best_obj_value = min(val[0] for val in enumeration_plans.values())
    else:
        best_obj_value = max(val[0] for val in enumeration_plans.values())
    print(f"best_obj_value {obj_type}: {best_obj_value}")

    best_plan_index = None
    best_dev = float('inf')

    for i, (val, dev) in enumeration_plans.items():
        if val == best_obj_value and dev < best_dev:
            best_plan_index = i
            best_dev = dev

    if best_plan_index is not None:
        best_entry = (new_plans[best_plan_index], [best_obj_value, best_obj_value], best_dev)

        # Check if an entry with the same deviation already exists
        existing_index = next((i for i, entry in enumerate(result) if math.isclose(entry[2], best_dev)), None)

        if existing_index is not None:
            result[existing_index] = best_entry
            print(f"Replaced plan at index {existing_index} with new plan {best_plan_index} (dev={best_dev}, {best_obj_value} {obj_type})")
        else:
            result.append(best_entry)
            print(f"Added plan {best_plan_index} with {best_obj_value} {obj_type} and deviation {best_dev} to result")

    if best_dev < epsilon:
        print("Deviation is too small, stopping early.")
        break

    min_deviation = best_dev - epsilon

In [ ]:
epsilon = 1 / (2 * G._k)
min_deviation = min(round(r[2], 1) for r in result)

if min_deviation < epsilon:
    no_solution_region = None
else:
    no_solution_region = [0, min_deviation]

if no_solution_region is not None:
    print(f"No feasible solution was found within the region: {no_solution_region}")

In [ ]:
from pareto import plot_pareto_frontiers

plot_pareto_frontiers(
                G,
                method='epsilon_constraint_method',
                plans=None,                                   #if method ='epsilon_constraint_method' is None 
                obj_types=obj_type,                               
                ideal_population=G._ideal_population,
                state=state,
                filepath=filepath,
                filename2=filename2,
                no_solution_region=no_solution_region,
                year=year,
                result=result                               #if method ='heuristic' is None 
             )

In [ ]:
from draw import draw_plan

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: int(x)}
G._L = 0
G._U = G._k * G._ideal_population

for plan, obj_bound, dev in result:
    obs_dev = observed_deviation_persons(G, plan, G._ideal_population)
    obj = compute_obj(G, plan, obj_type)
    obj_val = format_obj.get(obj_type, lambda x: round(x, 4))(obj)
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    title = f"{round(obs_dev, 2)}-person deviation ({deviation_percentage}%), with {obj_val} {obj_type}"
    draw_plan(filepath, filename2, G, plan, title=title, year=year)